In [1]:
# generate data
import Distributions
using Random

Random.seed!(42)

n_samples = 10
dimensionality = 2
order = dimensionality
rθ = [-0.5, 0.2]
inputs = randn(n_samples, dimensionality)
σ(w, x) = 1/(1+exp(-w'x))
πs = [σ(rθ, inputs[i, :]) for i in 1:n_samples]
outputs = [rand(Distributions.Bernoulli(πs[i])) for i in 1:n_samples]

10-element Vector{Bool}:
 1
 1
 1
 1
 0
 1
 1
 0
 1
 1

In [2]:
# test nonlinear
using Revise
using ForneyLab
import ForneyLab: unsafeMean, unsafeCov

graph = FactorGraph()

T = n_samples
x = Vector{Variable}(undef, T)
y = Vector{Variable}(undef, T)

# Hearing aid parameters
@RV θ  ~ GaussianMeanPrecision(placeholder(:m_θ, dims=(order,)), placeholder(:W_θ, dims=(order, order)))
f(w,x) = 1/(1+exp(-w'x))
for i in 1:T
    @eval $(Symbol("func$i"))(θ) = f(θ,inputs[$i, :])
    @RV x[i] ~ Nonlinear{Sampling}(θ, g=eval(Symbol("func$i")), in_variates=[Multivariate], out_variate=Univariate)
    @RV y[i] ~ Bernoulli(x[i])
    placeholder(y[i], :y, index=i)
end

In [3]:
algo = messagePassingAlgorithm(θ, free_energy=true)
src_code = algorithmSourceCode(algo, free_energy=true);

In [4]:
println(src_code);

begin

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 58))

messages[1] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=data[:m_θ]), Message(MatrixVariate, PointMass, m=data[:W_θ]))
messages[2] = ruleSPBernoulliIn1PN(Message(Univariate, PointMass, m=data[:y][10]), nothing)
messages[3] = ruleSPNonlinearSIn1MN(func10, messages[2], nothing, variate=Multivariate)
messages[4] = ruleSPBernoulliIn1PN(Message(Univariate, PointMass, m=data[:y][9]), nothing)
messages[5] = ruleSPNonlinearSIn1MN(func9, messages[4], nothing, variate=Multivariate)
messages[6] = ruleSPEqualityFn(messages[5], nothing, messages[3])
messages[7] = ruleSPBernoulliIn1PN(Message(Univariate, PointMass, m=data[:y][8]), nothing)
messages[8] = ruleSPNonlinearSIn1MN(func8, messages[7], nothing, variate=Multivariate)
messages[9] = ruleSPEqualityFn(messages[8], nothing, messages[6])
messages[10] = ruleSPBernoulliIn1PN(Message(Univariate, PointMass,

In [5]:
# Load algorithm
eval(Meta.parse(src_code))

freeEnergy (generic function with 1 method)

In [6]:
data = Dict(:y => outputs, :m_θ => zeros(dimensionality), :W_θ => 0.1*diageye(dimensionality))
marginals = step!(data)

Dict{Any, Any} with 11 entries:
  :x_3  => SampleList(s=[0.66, 0.61, 0.67, 0.65, 0.49, 0.53, 0.62, 0.44, 0.67, …
  :x_10 => SampleList(s=[0.02, 5.62e-03, 0.14, 4.36e-03, 0.02, 2.25e-03, 0.01, …
  :x_2  => SampleList(s=[0.64, 0.65, 0.65, 0.67, 0.65, 0.40, 0.80, 0.50, 0.57, …
  :x_5  => SampleList(s=[0.20, 0.26, 0.01, 0.04, 0.28, 0.15, 0.61, 0.04, 0.02, …
  :x_1  => SampleList(s=[0.85, 0.79, 0.73, 0.94, 0.23, 0.75, 0.86, 0.84, 0.95, …
  :x_4  => SampleList(s=[0.96, 0.83, 0.43, 0.50, 0.95, 0.72, 0.07, 0.57, 0.86, …
  :x_9  => SampleList(s=[0.36, 0.92, 1.00, 1.00, 0.99, 0.01, 1.00, 0.96, 0.93, …
  :θ    => 𝒩(m=[-1.33, 0.81], w=[[1.08, -0.08][-0.08, 1.36]])…
  :x_6  => SampleList(s=[0.55, 0.01, 0.02, 0.66, 0.25, 0.63, 0.85, 0.02, 0.71, …
  :x_7  => SampleList(s=[0.75, 0.84, 0.87, 0.54, 0.81, 0.87, 0.64, 0.91, 0.75, …
  :x_8  => SampleList(s=[0.14, 0.39, 0.18, 0.32, 0.24, 0.68, 0.84, 0.13, 0.45, …

In [7]:
meθ = unsafeMean(marginals[:θ])

2-element Vector{Float64}:
 -1.3319520146225503
  0.8149393727598759

In [8]:
weθ = unsafeCov(marginals[:θ])

2×2 Matrix{Float64}:
 0.929187   0.0559767
 0.0559767  0.737735

In [9]:
println("training errors = $(sum(abs.([round(f(meθ, inputs[i, :])) - round(πs[i]) for i in 1:n_samples])))")

training errors = 0.0


In [10]:
# test nonlinear
using Revise
using ForneyLab
using LinearAlgebra
import ForneyLab: unsafeMean, unsafeCov

graph = FactorGraph()

T = 2
x = Vector{Variable}(undef, T)
z = Vector{Variable}(undef, T)
y = Vector{Variable}(undef, T)

# Hearing aid parameters
@RV θ  ~ GaussianMeanPrecision(placeholder(:m_θ, dims=(dimensionality,)), placeholder(:W_θ, dims=(dimensionality, dimensionality)))
f(w, x) = 1/(1+exp(-w'x))
for i in 1:T
    @RV z[i] ~ GaussianMeanPrecision(inputs[i, :], 1e4*diageye(dimensionality))
    @RV x[i] ~ Nonlinear{Sampling}(θ, z[i], g=f, in_variates=[Multivariate, Multivariate], out_variate=Univariate)
    @RV y[i] ~ Bernoulli(x[i])
    placeholder(y[i], :y, index=i)
end

In [11]:
draw()

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.47.2 (20210527.0053)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 2929344861303102662 -->
 
 2929344861303102662 
 
 𝒩 
 gaussianmeanprecision_1 
 
<!-- 9643130187140608370 -->
 
 9643130187140608370 
 
 placeholder_W_θ 
 
<!-- 2929344861303102662--9643130187140608370 -->
 
 2929344861303102662--9643130187140608370 
 
 W_θ 
 1 out 
 3 w 
 
<!-- 17707422377843947055 -->
 
 17707422377843947055 
 
 placeholder_m_θ 
 
<!-- 2929344861303102662--17707422377843947055 -->
 
 2929344861303102662--17707422377843947055 
 
 m_θ 
 1 out 
 2 m 
 
<!-- 7208174809755892839 -->
 
 7208174809755892839 
 
 clamp_4 
 
<!-- 4655151314031304132 -->
 
 4655151314031304132 
 
 clamp_2 
 
<!-- 8177849662292221457 -->
 
 8177849662292221457 
 
 g{Sampling} 
 nonlinear_1 
 
<!-- 7051081533820749676 -->
 
 7051081533820749676 
 
 𝒩 
 gaussianmeanprecision_2 
 
<!-- 8177849662292221457--7051081533820749676 -->
 
 8177849662292221457--7051081533820749676 
 
 z_1 
 1 out 
 3 in2 
 
<!-- 3956432887097769494 -->
 
 3956432887097769494 
 
 clamp_1 
 
<!-- 11255939621104102397 -->
 
 11255939621104102397 
 
 Ber 
 bernoulli_2 
 
<!-- 112209210487119503 -->
 
 112209210487119503 
 
 g{Sampling} 
 nonlinear_2 
 
<!-- 11255939621104102397--112209210487119503 -->
 
 11255939621104102397--112209210487119503 
 
 x_2 
 1 out 
 2 p 
 
<!-- 3105371082948783497 -->
 
 3105371082948783497 
 
 Ber 
 bernoulli_1 
 
<!-- 3105371082948783497--8177849662292221457 -->
 
 3105371082948783497--8177849662292221457 
 
 x_1 
 1 out 
 2 p 
 
<!-- 3107973171706399681 -->
 
 3107973171706399681 
 
 = 
 equ_θ_1 
 
<!-- 3107973171706399681--2929344861303102662 -->
 
 3107973171706399681--2929344861303102662 
 
 θ 
 1 out 
 1 1 
 
<!-- 3107973171706399681--8177849662292221457 -->
 
 3107973171706399681--8177849662292221457 
 
 θ 
 2 in1 
 2 2 
 
<!-- 3107973171706399681--112209210487119503 -->
 
 3107973171706399681--112209210487119503 
 
 θ 
 2 in1 
 3 3 
 
<!-- 9129987650567176776 -->
 
 9129987650567176776 
 
 clamp_3 
 
<!-- 10369311190857547765 -->
 
 10369311190857547765 
 
 𝒩 
 gaussianmeanprecision_3 
 
<!-- 10369311190857547765--7208174809755892839 -->
 
 10369311190857547765--7208174809755892839 
 
 clamp_4 
 1 out 
 3 w 
 
<!-- 10369311190857547765--9129987650567176776 -->
 
 10369311190857547765--9129987650567176776 
 
 clamp_3 
 1 out 
 2 m 
 
<!-- 112209210487119503--10369311190857547765 -->
 
 112209210487119503--10369311190857547765 
 
 z_2 
 1 out 
 3 in2 
 
<!-- 7051081533820749676--4655151314031304132 -->
 
 7051081533820749676--4655151314031304132 
 
 clamp_2 
 1 out 
 3 w 
 
<!-- 7051081533820749676--3956432887097769494 -->
 
 7051081533820749676--3956432887097769494 
 
 clamp_1 
 1 out 
 2 m 
 
<!-- 7099486234168610497 -->
 
 7099486234168610497 
 
 placeholder_y_2 
 
<!-- 7099486234168610497--11255939621104102397 -->
 
 7099486234168610497--11255939621104102397 
 
 y_2 
 1 out 
 1 out 
 
<!-- 215911138147715210 -->
 
 215911138147715210 
 
 placeholder_y_1 
 
<!-- 215911138147715210--3105371082948783497 -->
 
 215911138147715210--3105371082948783497 
 
 y_1 
 1 out 
 1 out

In [12]:
# Define posterior factorization
pfz = PosteriorFactorization()

PosteriorFactorization(FactorGraph(Dict{Symbol, FactorNode}(:nonlinear_1 => Nonlinear{Sampling} with id nonlinear_1
, :equ_θ_1 => Equality with id equ_θ_1
, :clamp_3 => Clamp{Multivariate} with id clamp_3
, :placeholder_W_θ => Clamp{MatrixVariate} with id placeholder_W_θ
, :clamp_1 => Clamp{Multivariate} with id clamp_1
, :bernoulli_1 => Bernoulli with id bernoulli_1
, :gaussianmeanprecision_1 => GaussianMeanPrecision with id gaussianmeanprecision_1
, :gaussianmeanprecision_2 => GaussianMeanPrecision with id gaussianmeanprecision_2
, :clamp_2 => Clamp{MatrixVariate} with id clamp_2
, :placeholder_y_2 => Clamp{Univariate} with id placeholder_y_2
…), Edges:
Edge belonging to variable m_θ: ( placeholder_m_θ.i[out] )----( gaussianmeanprecision_1.i[m] ).
Edge belonging to variable W_θ: ( placeholder_W_θ.i[out] )----( gaussianmeanprecision_1.i[w] ).
Edge belonging to variable θ: ( gaussianmeanprecision_1.i[out] )----( equ_θ_1.i[1] ).
Edge belonging to variable clamp_1: ( clamp_1.i[out] )----

In [13]:
# Compile algorithm
algo = messagePassingAlgorithm(free_energy=true)

# Generate source code
src_code = algorithmSourceCode(algo, free_energy=true);

In [14]:
println(src_code)

begin

function init()

messages = Array{Message}(undef, 14)

messages[1] = Message(vague(GaussianMeanPrecision))
messages[2] = Message(vague(GaussianMeanPrecision))
messages[6] = Message(vague(GaussianWeightedMeanPrecision))
messages[9] = Message(vague(GaussianWeightedMeanPrecision))

return messages

end

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 14))

messages[1] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=[-0.444383357109696, 0.18702790710363]), Message(MatrixVariate, PointMass, m=Diagonal([10000.0, 10000.0])))
messages[2] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=[-0.5560268761463861, 1.0823812056084292]), Message(MatrixVariate, PointMass, m=Diagonal([10000.0, 10000.0])))
messages[3] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=data[:m_θ]), Message(MatrixVariate, PointMass, m=data[:W_θ]))
messages[4] = ruleSPBernoulli

In [15]:
# Load algorithm
eval(Meta.parse(src_code))

freeEnergy (generic function with 1 method)

In [16]:
ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=[0.15614346264074028, -1.590579974922555]), Message(MatrixVariate, PointMass, m=Diagonal([10000.0, 10000.0])))

Message: 𝒩(m=[0.16, -1.59], w=diag[1.00e+04, 1.00e+04])


In [26]:
function init()
    messages = Array{Message}(undef, 14)
    for i in 1:length(messages)
        messages[i] = Message(vague(GaussianMeanPrecision, 2))
    end
    messages
end

init (generic function with 1 method)

In [28]:
data = Dict(:y => outputs, :m_θ => zeros(dimensionality), :W_θ => 0.1*diageye(dimensionality))
messages = init()
marginals = Dict()
step!(data, marginals, messages)

LoadError: MethodError: no method matching ruleSPNonlinearSOutNGX(::typeof(f), ::Nothing, ::Message{GaussianWeightedMeanPrecision, Multivariate}, ::Message{GaussianMeanPrecision, Multivariate}; variate=Univariate)
[0mClosest candidates are:
[0m  ruleSPNonlinearSOutNGX(::Function, ::Nothing, ::Message{var"#s103", V} where var"#s103"<:Gaussian...; n_samples) where V<:VariateType at In[27]:4[91m got unsupported keyword argument "variate"[39m
[0m  ruleSPNonlinearSOutNGX(::Function, ::Nothing, ::Message{var"#s148", var"#s147"} where {var"#s148"<:Gaussian, var"#s147"<:VariateType}...; n_samples, variate) at /Users/apodusenko/.julia/dev/ForneyLab/src/engines/julia/update_rules/nonlinear_sampling.jl:72

In [383]:
messages

14-element Vector{Message}:
    Message: 𝒩(m=[-0.44, 0.19], w=diag[1.00e+04, 1.00e+04])

    Message: 𝒩(m=[-0.56, 1.08], w=diag[1.00e+04, 1.00e+04])

    Message: 𝒩(m=[0.00, 0.00], w=diag[0.10, 0.10])

    Message: Beta(a=2, b=1)

 #undef
    Message: 𝒩(xi=0.00, w=1.00e-12)

 #undef
 #undef
    Message: 𝒩(xi=0.00, w=1.00e-12)

 #undef
 #undef
 #undef
 #undef
 #undef

In [391]:
messages[1] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=[-0.444383357109696, 0.18702790710363]), Message(MatrixVariate, PointMass, m=Diagonal([10000.0, 10000.0])))
messages[2] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=[-0.5560268761463861, 1.0823812056084292]), Message(MatrixVariate, PointMass, m=Diagonal([10000.0, 10000.0])))
messages[3] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=data[:m_θ]), Message(MatrixVariate, PointMass, m=data[:W_θ]))
messages[4] = ruleSPBernoulliIn1PN(Message(Univariate, PointMass, m=data[:y][1]), nothing)
messages[5] = ruleSPNonlinearSInGX(f, 1, messages[4], messages[9], messages[2], variate=Multivariate)
messages[6] = ruleSPEqualityGaussian(messages[3], messages[5], nothing)
messages[7] = ruleSPBernoulliIn1PN(Message(Univariate, PointMass, m=data[:y][2]), nothing)
messages[8] = ruleSPNonlinearSInGX(f, 1, messages[7], messages[6], messages[1], variate=Multivariate)
messages[9] = ruleSPEqualityGaussian(messages[3], nothing, messages[8])
messages[10] = ruleSPEqualityGaussian(nothing, messages[5], messages[8])
messages[11] = ruleSPNonlinearSInGX(f, 2, messages[7], messages[6], messages[1], variate=Multivariate)
messages[12] = ruleSPNonlinearSInGX(f, 2, messages[4], messages[9], messages[2], variate=Multivariate)
messages[13] = ruleSPNonlinearSOutNGX(f, nothing, messages[6], messages[1], variate=Univariate)

LoadError: MethodError: no method matching ruleSPNonlinearSOutNGX(::typeof(f), ::Nothing, ::Message{GaussianWeightedMeanPrecision, Multivariate}, ::Message{GaussianMeanPrecision, Multivariate}; variate=Univariate)
[0mClosest candidates are:
[0m  ruleSPNonlinearSOutNGX(::Function, ::Nothing, ::Message{var"#s158", V} where var"#s158"<:Gaussian...; n_samples) where V<:ForneyLab.VariateType at /Users/apodusenko/.julia/dev/ForneyLab/src/engines/julia/update_rules/nonlinear_sampling.jl:73[91m got unsupported keyword argument "variate"[39m
[0m  ruleSPNonlinearSOutNGX(::Function, ::Nothing, ::Message{var"#s148", var"#s147"} where {var"#s148"<:Gaussian, var"#s147"<:ForneyLab.VariateType}...; n_samples, variate) at /Users/apodusenko/.julia/dev/ForneyLab/src/engines/julia/update_rules/nonlinear_sampling.jl:65